# 4. GPU

https://towardsdatascience.com/installing-pytorch-on-apple-m1-chip-with-gpu-acceleration-3351dc44d67c

In [1]:
import time as time

import torch
from torch import nn

In [2]:
#this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())

True


In [3]:
#this ensures that the current current PyTorch installation was built with MPS activated
print(torch.backends.mps.is_built())

True


We first need to define the device we are using:

In [4]:
device = torch.device("mps")

Then, we try some calculations by storing the tensors on our GPU:

In [5]:
dtype = torch.float

#generate random data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

#randomly initialize weights
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

NameError: name 'math' is not defined

In [15]:
learning_rate = 1e-6

for t in range(2000):
    #forward propagation
    y_pred = a + b * x + c * x ** 2 + d * x ** 3
    #compute loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)
    
    #compute gradients
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    #update weights
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 836.641845703125
199 556.23291015625
299 370.8101806640625
399 248.19534301757812
499 167.11306762695312
599 113.49539184570312
699 78.03921508789062
799 54.59271240234375
899 39.08808135986328
999 28.835145950317383
1099 22.054960250854492
1199 17.571271896362305
1299 14.606252670288086
1399 12.645549774169922
1499 11.348925590515137
1599 10.491454124450684
1699 9.924413681030273
1799 9.549428939819336
1899 9.30142593383789
1999 9.137421607971191
Result: y = -0.0008230566745623946 + 0.8393610119819641 x + 0.00014198923599906266 x^2 + -0.09085827320814133 x^3


We can also store the parameters of a model on the GPU:

In [6]:
X = torch.randn(100000, 10000, device=device, dtype=dtype)
X

/Users/ryanl/opt/anaconda3/lib/python3.9/site-packages/torch/_tensor_str.py:115: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(


tensor([[-1.2767,  0.3843,  1.0527,  ...,  1.6313, -1.0999, -1.3992],
        [ 0.2610,  0.6141, -1.0926,  ..., -0.5260, -0.1930,  0.5506],
        [ 0.1557, -0.0732, -1.7974,  ...,  0.9506, -0.5124,  0.3485],
        ...,
        [ 1.0980,  0.4339, -0.0071,  ...,  0.1427,  1.0008,  0.8351],
        [-0.3857, -1.2685,  0.5023,  ...,  0.2298,  1.7227,  1.6923],
        [-0.4016,  0.7939,  0.0274,  ..., -0.1553,  0.2953,  0.1715]],
       device='mps:0')

In [7]:
net = nn.Sequential(nn.Linear(10000, 128),
                    nn.ReLU(),
                    nn.Linear(128, 32),
                    nn.ReLU(),
                    nn.Linear(32, 1),
                    )
net = net.to(device=device)

In [8]:
s = time.time()
net(X)
e = time.time()
e - s

0.058466196060180664

In [9]:
X = torch.randn(100000, 10000)
X

tensor([[-0.8059, -0.6013,  0.4279,  ...,  1.5531, -0.2329, -0.8950],
        [-0.5878,  1.1377, -1.3780,  ...,  0.2720, -0.5065,  0.6078],
        [ 0.8171, -0.0625,  0.5603,  ..., -0.9016, -0.8044,  1.8955],
        ...,
        [ 0.7844,  1.0518, -0.7314,  ..., -0.7668, -0.4212,  0.2655],
        [ 0.6507, -0.9097,  1.7279,  ...,  0.4537, -0.4229,  0.9502],
        [ 0.2542, -0.3979,  1.3094,  ..., -0.4925,  0.3084, -0.4811]])

In [10]:
net2 = nn.Sequential(nn.Linear(10000, 128),
                    nn.ReLU(),
                    nn.Linear(128, 32),
                    nn.ReLU(),
                    nn.Linear(32, 1),
                    )

In [11]:
s = time.time()
net2(X)
e = time.time()
e - s

0.8828158378601074